# Calculate Highly Variable Features And Get mC Fraction AnnData

## Purpose
The purpose of this step is to select highly variable features (HVF) and generate cell-by-feature methylation fraction matrix for clustering. The highly variable features are selected by comparing feature's normalized dispersion among cells.

## Input
- Filtered cell metadata;
- MCDS files;
- Feature list from basic feature filtering

## Output
- cell-by-HVF methylation fraction matrix stored in AnnData format, e.g., mCH adata and mCG adata.

## Import

In [1]:
import yaml
import pandas as pd
import dask
from ALLCools.mcds import MCDS
from wmb import cemba

## Parameters

In [2]:
with open('config/03a.yaml', 'r') as f:
    config = yaml.safe_load(f)
    locals().update(config)
    print('Notebook configs:')
    for _k, _v in config.items():
        print(f'{_k} = {_v}')

Notebook configs:
dataset = mC
mch_pattern = CHN
select_cells = select_cells.txt
var_dim = chrom100k


In [3]:
# Parameters
dataset = "mC"
mch_pattern = "CHN"
select_cells = "select_cells.txt"
var_dim = "chrom100k"


In [4]:
if dataset == 'mC':
    mcds_path = cemba.CEMBA_SNMC_MCDS_PATH
else:
    mcds_path = cemba.CEMBA_SNM3C_MCDS_PATH

## Load Data

### Metadata

In [5]:
if dataset == 'mC':
    metadata = cemba.get_mc_mapping_metric(select_cells=select_cells)
else:
    metadata = cemba.get_m3c_mapping_metric(select_cells=select_cells)

total_cells = metadata.shape[0]
print(f'Metadata of {total_cells} cells')

Metadata of 12291 cells


In [6]:
metadata.head()

,mCCCFrac,mCGFrac,mCHFrac,FinalmCReads,DissectionRegion,Plate,Col384,Row384,Slice,Sample,Technology,InputReads,PassBasicQC,PlateNormCov
cell,,,,,,,,,,,,,,
14B_M_0,0.006819,0.735136,0.012240,1061284.0,14B,CEMBA210617_14B_1,9,0,14,14B_210617,snmC-seq3,2643012,True,0.945824
14B_M_1,0.007127,0.750066,0.013914,1059639.0,14B,CEMBA210617_14B_1,20,0,14,14B_210617,snmC-seq3,2658348,True,0.944358
14B_M_2,0.007187,0.736750,0.014544,773689.0,14B,CEMBA210617_14B_1,21,0,14,14B_210617,snmC-seq3,1827306,True,0.689518
14B_M_3,0.008106,0.748119,0.014039,1567001.0,14B,CEMBA210617_14B_1,8,0,14,14B_210617,snmC-seq3,3767798,True,1.396523
14B_M_4,0.007109,0.758576,0.021379,1121801.0,14B,CEMBA210617_14B_1,9,1,14,14B_210617,snmC-seq3,3119214,True,0.999758


### MCDS

In [7]:
with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    # still use all the cells to load MCDS
    total_mcds = MCDS.open(mcds_path,
                           var_dim=var_dim,
                           use_obs=metadata.index)

## Add mC Rate

In [8]:
total_mcds.add_mc_rate(var_dim=var_dim,
                       normalize_per_cell=True,
                       clip_norm_value=10)

total_mcds

<xarray.MCDS>
Dimensions:             (cell: 12291, chrom100k: 27269, count_type: 2,
                         mc_type: 2)
Coordinates:
  * cell                (cell) <U15 '17A_M_2553' '15B_M_3042' ... '17A_M_2837'
  * chrom100k           (chrom100k) <U10 'chr1_0' 'chr1_1' ... 'chrY_917'
    chrom100k_chrom     (chrom100k) <U5 dask.array<chunksize=(4096,), meta=np.ndarray>
    chrom100k_cov_mean  (chrom100k) float64 dask.array<chunksize=(4096,), meta=np.ndarray>
    chrom100k_end       (chrom100k) int64 dask.array<chunksize=(4096,), meta=np.ndarray>
    chrom100k_start     (chrom100k) int64 dask.array<chunksize=(4096,), meta=np.ndarray>
  * count_type          (count_type) <U3 'mc' 'cov'
  * mc_type             (mc_type) <U3 'CHN' 'CGN'
Data variables:
    chrom100k_da        (cell, chrom100k, mc_type, count_type) uint16 dask.array<chunksize=(2458, 4096, 1, 1), meta=np.ndarray>
    chrom100k_da_frac   (cell, chrom100k, mc_type) float64 dask.array<chunksize=(2458, 4096, 1), meta=np.ndarray>
Attributes:
    obs_dim:  cell
    var_dim:  chrom100k

## Save AnnData

In [9]:
mch_hvf = pd.read_hdf('mch_hvf.hdf')

feature_select = mch_hvf['feature_select']
total_mcds.coords[f'{var_dim}_CHN_feature_select'] = feature_select

In [10]:
mch_adata = total_mcds.get_adata(mc_type=mch_pattern,
                                 var_dim=var_dim,
                                 select_hvf=True)

mch_adata.write_h5ad(f'mCH.HVF.h5ad')

mch_adata

/home/hanliu/miniconda3/envs/allcools_new/lib/python3.8/site-packages/dask/core.py:119: RuntimeWarning: invalid value encountered in true_divide
  return func(*(_execute_task(a, cache) for a in args))


/home/hanliu/miniconda3/envs/allcools_new/lib/python3.8/site-packages/anndata/_core/anndata.py:1228: FutureWarning: The `inplace` parameter in pandas.Categorical.reorder_categories is deprecated and will be removed in a future version. Reordering categories will always return a new Categorical object.
  c.reorder_categories(natsorted(c.categories), inplace=True)
... storing 'chrom' as categorical


AnnData object with n_obs × n_vars = 12291 × 15000
    var: 'chrom', 'cov_mean', 'end', 'start', 'CHN_feature_select'